In [1]:
import os
import subprocess
from tifffile import imread, imwrite
import numpy as np

In [2]:
input_dir = r'R:\Aidan\ULA_timecourse_ARC_006_temp'
output_dir = r'R:\Aidan\ULA_timecourse_ARC_006_movies'
os.makedirs(output_dir, exist_ok=True)

BIT_DEPTH = np.uint16
CREATE_SMALL_MOVIES = True
SMALL_FACTOR = 4

In [3]:
#############################################################
# specify which images to re-stitch in the brute force way
#############################################################

im_list = ['0944S']
n_rows = 5
n_columns = 5

In [5]:
def merge_images(img_list, n_rows, n_columns, overlap = 0.1, orientation = 'horizontal', direction = 'snake'):

    if orientation == 'horizontal' and direction == 'snake':

        if n_rows * n_columns != len(img_list):
            raise ValueError('Number of images does not match the number of rows and columns')

        # create an empty frame
        im_size_row= img_list[0].shape[0] * n_rows - int(overlap * img_list[0].shape[0] * (n_rows - 1))
        im_size_col = img_list[0].shape[1] * n_columns - int(overlap * img_list[0].shape[1] * (n_columns - 1))

        frame = np.zeros((im_size_row, im_size_col), dtype = img_list[0].dtype)

        # populate the frame 
        for i, img in enumerate(img_list):

            row = i // n_columns
            col = i % n_columns

            if row % 2 != 0:
                col = n_columns - col - 1

            tile = frame[row * img.shape[0] - int(row * overlap * img.shape[0]):(row + 1) * img.shape[0] - int(row * overlap * img.shape[0]), 
                    col * img.shape[1] - int(col * overlap * img.shape[1]):(col + 1) * img.shape[1] - int(col * overlap * img.shape[1])]
            
            tile = np.max(np.array([tile,img]), axis = 0)
                
            frame[row * img.shape[0] - int(row * overlap * img.shape[0]):(row + 1) * img.shape[0] - int(row * overlap * img.shape[0]), 
                    col * img.shape[1] - int(col * overlap * img.shape[1]):(col + 1) * img.shape[1] - int(col * overlap * img.shape[1])] = tile  
          
        return frame

    else:
        raise ValueError('Only snake direction implemented for now')

In [6]:
for label in im_list:

    print(f'Processing {label}...')

    ch_list = set([x.split('_')[2] for x in os.listdir(output_dir) if f'{label}_ch' in x and '_m_' in x])
    t_list = sorted(list(set([x.split('_')[6][:-4] for x in os.listdir(output_dir) if f'{label}_ch' in x and '_m_' in x])))

    for ch in ch_list:

        print(f'Processing channel {ch}...')

        merged_im_list = []
        for t in t_list:

            print(f'Processing timepoint {t}...')

            file_list = sorted([x for x in os.listdir(output_dir) if f'{label}_ch_{ch}' in x and f'{t}.tif' in x])
            img_list = [imread(os.path.join(output_dir, x)) for x in file_list]

            # merge images
            frame = merge_images(img_list, n_rows, n_columns)

            merged_im_list.append(frame)
            
        # movie pathway
        save_path = os.path.join(output_dir,label + '_ch_' + ch + '.tif')

        im_to_save = np.array(merged_im_list).astype(BIT_DEPTH)
        imwrite(save_path, im_to_save, imagej=True)

        #create a small movie for quick inspection
        if CREATE_SMALL_MOVIES:
            save_path_small = os.path.join(output_dir,label + '_ch_' + str(ch).zfill(2) + '_small.tif')
            imwrite(save_path_small, im_to_save[:,::SMALL_FACTOR,::SMALL_FACTOR], imagej=True)


Processing 0944S...
Processing channel 00...
Processing timepoint 000...
Processing timepoint 001...
Processing timepoint 002...
Processing timepoint 003...
Processing timepoint 004...
Processing timepoint 005...
Processing timepoint 006...
Processing timepoint 007...
Processing timepoint 008...
Processing timepoint 009...
Processing timepoint 010...
Processing timepoint 011...
Processing timepoint 012...
Processing timepoint 013...
Processing timepoint 014...
Processing timepoint 015...
Processing timepoint 016...
Processing timepoint 017...
Processing timepoint 018...
Processing timepoint 019...
Processing timepoint 020...
Processing timepoint 021...
Processing timepoint 022...
Processing timepoint 023...
Processing timepoint 024...
Processing timepoint 025...
Processing timepoint 026...
Processing timepoint 027...


c:\Users\KMK280\AppData\Local\anaconda3\envs\napari-env\lib\site-packages\tifffile\tifffile.py:3474: UserWarning: <tifffile.TiffWriter '0944S_ch_00.tif'> truncating ImageJ file
  warnings.warn(


Processing channel 01...
Processing timepoint 000...
Processing timepoint 001...
Processing timepoint 002...
Processing timepoint 003...
Processing timepoint 004...
Processing timepoint 005...
Processing timepoint 006...
Processing timepoint 007...
Processing timepoint 008...
Processing timepoint 009...
Processing timepoint 010...
Processing timepoint 011...
Processing timepoint 012...
Processing timepoint 013...
Processing timepoint 014...
Processing timepoint 015...
Processing timepoint 016...
Processing timepoint 017...
Processing timepoint 018...
Processing timepoint 019...
Processing timepoint 020...
Processing timepoint 021...
Processing timepoint 022...
Processing timepoint 023...
Processing timepoint 024...
Processing timepoint 025...
Processing timepoint 026...
Processing timepoint 027...


c:\Users\KMK280\AppData\Local\anaconda3\envs\napari-env\lib\site-packages\tifffile\tifffile.py:3474: UserWarning: <tifffile.TiffWriter '0944S_ch_01.tif'> truncating ImageJ file
  warnings.warn(
